In [5]:
#5West_Bengal_Transport_corporation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from decimal import Decimal
import pandas as pd
import time


driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20) 

bus_route_link = []
bus_route_name = []
bus_route_final = []
#bus_route = []
bus_names = [] 
bus_type = [] 
departing_time = []
duration = [] 
Reaching_time = [] 
star_rating = [] 
price = [] 
seat_availablity = []

def jump_pages():
    for page_number in range(1,3):
        if page_number < 3:
            try:
                find_bus_routes = driver.find_elements(By.CLASS_NAME, 'route')
                for route in find_bus_routes:
                    try:
                        route_name = route.get_attribute("title")
                        route_link = route.get_attribute("href")
                        bus_route_name.append(route_name)
                        bus_route_link.append(route_link)
                        print(route_name, route_link)
                    except Exception as e:
                        print('Failure')
                
                # Handle pagination
                pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "DC_117_paginationTable")]')))
                next_page_button = pagination_container.find_element(By.XPATH, f'//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
                action = ActionChains(driver)
                action.move_to_element(next_page_button).perform()
                time.sleep(2)
                next_page_button.click()

            except Exception as e:
                print('Failure')
                break


def scroll(): # function to scroll page from top to bottom
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")# Scroll down to the bottom in order to load all the buses
        time.sleep(2)# Wait for page to load
        new_height = driver.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
        if new_height == last_height:# If it is the same height then it is at the end of the page
            break
        last_height = new_height

def click_button(): # function to click button for view buses for government buses
    view_buses = driver.find_elements(By.CSS_SELECTOR, '.p-left-10.icon.icon-down')
    if view_buses:
        button_count = len(view_buses)
        for i in range(button_count-1 , -1, -1):
            view_buses[i].click()
            time.sleep(2)

def get_bus_details(): # function to get all bus details
    rate = driver.find_elements(By.CSS_SELECTOR,".fare.d-block")#
    seat_available = driver.find_elements(By.CSS_SELECTOR,".column-eight.w-15.fl")
    rating = driver.find_elements(By.CSS_SELECTOR,".column-six.p-right-10.w-10.fl")
    names = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
    b_type = driver.find_elements(By.CSS_SELECTOR,".bus-type.f-12.m-top-16.l-color.evBus")
    start_time = driver.find_elements(By.CSS_SELECTOR,".dp-time.f-19.d-color.f-bold")
    total_time = driver.find_elements(By.CSS_SELECTOR,".dur.l-color.lh-24")
    end_time =  driver.find_elements(By.CSS_SELECTOR,".bp-time.f-19.d-color.disp-Inline")
    
    time.sleep(5)
    print(len(names),len(b_type),len(start_time),len(total_time),len(end_time),len(rating),len(rate),len(seat_available))

    for i in range(0,len(seat_available)):#(len(rating)):
        bus_names.append(names[i].text)
        bus_type.append(b_type[i].text)
        departing_time.append(start_time[i].text)
        duration.append(total_time[i].text)
        Reaching_time.append(end_time[i].text)
        if (rating[i].text) == "New":
            star_rating.append(0.0)
        else:
            star_rating.append((rating[i].text)[0:3])
        price.append((rate[i].text))
        seat_availablity.append((seat_available[i].text)[0:2])
    
    print(len(bus_names),len(bus_type),len(departing_time),len(duration),len(Reaching_time),len(star_rating),len(price),len(seat_availablity))
    print((bus_names),(bus_type),(departing_time),(duration),(Reaching_time),(star_rating),(price),(seat_availablity))


driver.get("https://www.redbus.in/online-booking/meghalaya-transport-corporation-mtc")
driver.maximize_window()
time.sleep(5)


jump_pages()
            
print("Total routes",len(bus_route_name))

for i in range (0,len(bus_route_link)):# used 2 just to check if every thing is working as expected
    driver.get(bus_route_link[i])
    body = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.clearfix.bus-item')))
    click_button()
    scroll()
    time.sleep(5)
    get_bus_details()
    temp_name = []
    temp_link = []
    bus_route = []
    state_service = []
    print(len(bus_names))
    for j in range(len(bus_names)):
        temp_name.append(bus_route_name[i])
        temp_link.append(bus_route_link[i])
        state_service.append("Mehalaya")
    print(len(temp_name),len(temp_link))
    bus_route = list(zip(state_service,temp_name,temp_link,bus_names,bus_type,departing_time,duration,Reaching_time,star_rating,price,seat_availablity))
    bus_route_final.extend(bus_route)
    print(i,bus_route_name[i],"completed")
    temp_name.clear()
    temp_link.clear()
    state_service.clear()
    bus_route.clear()
    bus_names.clear()
    bus_type.clear()
    departing_time.clear()
    duration.clear()
    Reaching_time.clear()
    star_rating.clear()
    price.clear()
    seat_availablity.clear()
    
df_bus_details = pd.DataFrame(bus_route_final, columns=["Service_Name","Route_name", "Route_link", "Bus_names","Bus_type","Departing_time","Duration","Reaching_time","Star_rating","Price","Seat_availablity"])
df_bus_details['Seat_availablity']=df_bus_details['Seat_availablity'].astype(int)
df_bus_details['Price'] = df_bus_details['Price'].str.replace('INR ', '', regex=False)
df_bus_details['Star_rating'] = df_bus_details['Star_rating'].str.replace(' ', '', regex=False)
df_bus_details['Price'] = df_bus_details['Price'].astype(float)
df_bus_details['Star_rating'] = df_bus_details['Star_rating'].fillna(0.0)

Guwahati to Shillong https://www.redbus.in/bus-tickets/guwahati-to-shillong
Shillong to Guwahati https://www.redbus.in/bus-tickets/shillong-to-guwahati
Tura (Meghalaya) to Shillong https://www.redbus.in/bus-tickets/tura-meghalaya-to-shillong
Shillong to Tura (Meghalaya) https://www.redbus.in/bus-tickets/shillong-to-tura-meghalaya
Shillong to Williamnagar (Meghalaya) https://www.redbus.in/bus-tickets/shillong-to-williamnagar-meghalaya
Williamnagar (Meghalaya) to Shillong https://www.redbus.in/bus-tickets/williamnagar-meghalaya-to-shillong
Shillong to Silchar https://www.redbus.in/bus-tickets/shillong-to-silchar
Phulbari to Shillong https://www.redbus.in/bus-tickets/phulbari-to-shillong
Shillong to Phulbari https://www.redbus.in/bus-tickets/shillong-to-phulbari
Shillong to Karimganj https://www.redbus.in/bus-tickets/shillong-to-karimganj
Silchar to Shillong https://www.redbus.in/bus-tickets/silchar-to-shillong
Karimganj to Shillong https://www.redbus.in/bus-tickets/karimganj-to-shillong


In [9]:
df_bus_details['Star_rating'] = df_bus_details['Star_rating'].astype(float)

In [11]:
df_bus_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Service_Name      35 non-null     object 
 1   Route_name        35 non-null     object 
 2   Route_link        35 non-null     object 
 3   Bus_names         35 non-null     object 
 4   Bus_type          35 non-null     object 
 5   Departing_time    35 non-null     object 
 6   Duration          35 non-null     object 
 7   Reaching_time     35 non-null     object 
 8   Star_rating       35 non-null     float64
 9   Price             35 non-null     float64
 10  Seat_availablity  35 non-null     int32  
dtypes: float64(2), int32(1), object(8)
memory usage: 3.0+ KB


In [13]:
df_bus_details.to_csv("Mehalaya.csv",index = False)